In [2]:
import pandas as pd
import numpy as np
import tensorflow as tf
from PIL import Image, ImageDraw, ImageFont
import matplotlib.pyplot as plt
import re

In [3]:
df = pd.read_csv('./datasets/ratings.txt', sep='\t', encoding='utf-8')
# df = df[['text', 'label']]
df = df.dropna()
df = df.reset_index(drop=True)
df = df[['document', 'label']]
df.columns=['text','label']

In [68]:
df = pd.read_csv('./datasets/hate_speech_binary_dataset.csv', encoding='utf-8')
# df = df[['text', 'label']]
df = df.dropna()
df = df.reset_index(drop=True)
df.columns=['text','label']

In [69]:
df

,text,label
0,정말 재밌다 연기도 좋고 디카프리오 짱,1
1,심쿵심쿵 미치네요이수혁땜에 잠 못자겠어요ㅠ자꾸 아른거림ㅠ,1
2,하지만 이니후빨러들은 이런거 관심 하나도 없음,0
3,@착한아이임당 A4 용지 덮고?,0
4,"진짜 평점 믿으면 안된 다는 걸 현실로 보여주는 영화입니다. 네티즌,전문가 하나같이...",1
...,...,...
189990,원작을 읽을 때 이런 건 절대 영상화하기 힘들다고 생각했는데 벤휘쇼의 연기와 더불어...,1
189991,케석대 어깨 올라간거봐라 ㅋㅋ,0
189992,@김짜꾸 day and night\n\nround the clock\n\nwitho...,0
189993,로버트다우니주니어를 좋아해서 봤는데너무재밌게 봤던영화생각없이 볼때 딱좋음,1


In [7]:
with open('./datasets/stopwords.txt', 'r') as f:
    lines = f.readlines()

# lines = [line.strip() for line in lines]
stopwords = list([line.strip() for line in lines if line != ""][1:])

In [8]:
stopwords

['아',
 '휴',
 '아이구',
 '아이쿠',
 '아이고',
 '어',
 '나',
 '우리',
 '저희',
 '따라',
 '의해',
 '을',
 '를',
 '에',
 '의',
 '가',
 '으로',
 '로',
 '에게',
 '뿐이다',
 '의거하여',
 '근거하여',
 '입각하여',
 '기준으로',
 '예하면',
 '예를 들면',
 '예를 들자면',
 '저',
 '소인',
 '소생',
 '저희',
 '지말고',
 '하지마',
 '하지마라',
 '다른',
 '물론',
 '또한',
 '그리고',
 '비길수 없다',
 '해서는 안된다',
 '뿐만 아니라',
 '만이 아니다',
 '만은 아니다',
 '막론하고',
 '관계없이',
 '그치지 않다',
 '그러나',
 '그런데',
 '하지만',
 '든간에',
 '논하지 않다',
 '따지지 않다',
 '설사',
 '비록',
 '더라도',
 '아니면',
 '만 못하다',
 '하는 편이 낫다',
 '불문하고',
 '향하여',
 '향해서',
 '향하다',
 '쪽으로',
 '틈타',
 '이용하여',
 '타다',
 '오르다',
 '제외하고',
 '이 외에',
 '이 밖에',
 '하여야',
 '비로소',
 '한다면 몰라도',
 '외에도',
 '이곳',
 '여기',
 '부터',
 '기점으로',
 '따라서',
 '할 생각이다',
 '하려고하다',
 '이리하여',
 '그리하여',
 '그렇게 함으로써',
 '하지만',
 '일때',
 '할때',
 '앞에서',
 '중에서',
 '보는데서',
 '으로써',
 '로써',
 '까지',
 '해야한다',
 '일것이다',
 '반드시',
 '할줄알다',
 '할수있다',
 '할수있어',
 '임에 틀림없다',
 '한다면',
 '등',
 '등등',
 '제',
 '겨우',
 '단지',
 '다만',
 '할뿐',
 '딩동',
 '댕그',
 '대해서',
 '대하여',
 '대하면',
 '훨씬',
 '얼마나',
 '얼마만큼',
 '얼마큼',
 '남짓',
 '여',
 '얼마간',
 '약간',
 '다소',
 '좀',
 '조

# Okt 형태소 분석 학습

In [10]:
from konlpy.tag import Okt
okt = Okt()

In [23]:
# sample = df['text'][199987]
sample = '재밌다 ㅋㅋㅋㅋ ㄱ\-ㅈ\- test.. !+ 테스트당'
words = okt.morphs(sample, norm=True)

In [24]:
words

['재밌다', 'ㅋㅋㅋ', 'ㄱ', '\\-', 'ㅈ', '\\-', 'test', '..', '!+', '테스트', '당']

In [25]:
processed_words = [word for word in words if word not in stopwords]

In [26]:
" ".join(processed_words)

'재밌다 ㅋㅋㅋ ㄱ \\- ㅈ \\- test .. !+ 테스트 당'

In [19]:
fasttext_data = './datasets/ratings_fasttext.txt'

In [20]:
# with open(fasttext_data, 'a', encoding='utf-8') as f:
#     for s in df['text']:
#         sentence = re.sub(r'[^가-힣\s]', '', s)
#         words = okt.morphs(sentence, norm=True)
#         processed_words = " ".join([word for word in words if word not in stopwords])
#         f.write(processed_words + '\n')

In [133]:
# with open(fasttext_data, 'w', encoding='utf-8') as f:
#     for s in df['text']:
#         sentence = re.sub(r'[^가-힣\s]', '', s)
#         words = okt.morphs(sentence, norm=True)
#         processed_words = " ".join([word for word in words if word not in stopwords])
#         f.write(processed_words + '\n')

# 학습

In [74]:
import fasttext

In [136]:
model = fasttext.train_unsupervised(input=fasttext_data, model='cbow')

Read 2M words
Number of words:  25669
Number of labels: 0
Progress: 100.0% words/sec/thread:  152335 lr:  0.000000 avg.loss:  2.277684 ETA:   0h 0m 0s


In [22]:
model = fasttext.train_unsupervised(input=fasttext_data, model='cbow')

Read 4M words
Number of words:  44620
Number of labels: 0
Progress: 100.0% words/sec/thread:  159975 lr:  0.000000 avg.loss:  2.040881 ETA:   0h 0m 0s


In [75]:
model = fasttext.train_unsupervised(input=jamo_fasttext_data, model='cbow')

Read 5M words
Number of words:  47149
Number of labels: 0
Progress: 100.0% words/sec/thread:  144790 lr:  0.000000 avg.loss:  1.897262 ETA:   0h 0m 0s


In [23]:
model.save_model("fasttext_model.bin")

In [76]:
model.save_model("fasttext_jamo_model.bin")

In [116]:
model = fasttext.load_model("fasttext_model.bin")

In [77]:
model = fasttext.load_model("fasttext_jamo_model.bin")

In [138]:
model['word']

array([-0.01376803,  0.02405382,  0.00689063,  0.00914073, -0.02817432,
       -0.02283239, -0.01213117,  0.01371536, -0.01318257, -0.03363256,
       -0.0161895 , -0.02667965, -0.01100159, -0.04583087, -0.03016288,
        0.00792429, -0.05242412,  0.04877637,  0.00342078, -0.02727258,
        0.00248586, -0.00687346, -0.00087517,  0.01480939,  0.02526407,
       -0.05215883,  0.01168032, -0.01802595,  0.02374715,  0.03216464,
        0.01770702, -0.05226377, -0.034553  , -0.02188332, -0.0377759 ,
       -0.02198618, -0.00057473, -0.03971213,  0.02086725,  0.01200611,
       -0.01229967,  0.0015644 , -0.0249375 , -0.02258019, -0.02497591,
       -0.03072441,  0.03733213,  0.02994373, -0.04566695,  0.00500638,
       -0.02630633,  0.00896328,  0.02000717,  0.07722417,  0.0056197 ,
       -0.02953898, -0.03807686,  0.07632416, -0.01463063,  0.02754509,
        0.03214902,  0.02673713,  0.04675517,  0.0600478 , -0.07335752,
        0.02287391, -0.05364009, -0.00042287,  0.04017952,  0.02

In [79]:
model[j2hcj(h2j('안녕'))]

array([ 0.6965292 , -0.21400873,  1.497063  ,  0.51645315,  0.78901595,
       -1.101479  , -0.18837924, -0.44369066, -0.89291173,  0.7180929 ,
       -0.3652067 ,  0.5292693 ,  0.6567712 ,  0.25390553, -0.31230664,
       -0.3781159 , -0.22069359, -0.85900146, -0.14592446,  0.5325652 ,
       -1.2292607 , -0.00560008, -0.7025034 , -0.64670324,  0.2679109 ,
        0.2630891 , -0.17430037,  0.21996473,  0.5044779 ,  0.52684945,
        0.02544295,  0.611828  , -0.02957369, -0.80398273,  0.9378988 ,
       -0.7147775 , -0.60836565,  0.0820001 ,  0.70859903,  0.48049796,
       -0.7796267 ,  0.43698823,  0.6253108 , -0.01020891, -1.4926325 ,
       -0.759376  ,  0.42834693,  1.1598433 , -1.1175762 ,  0.03844259,
        0.35297847,  0.7795538 ,  0.85395736,  0.75285614,  1.3735087 ,
       -1.1985319 ,  0.14883661,  0.21110861,  0.35560665, -0.5628659 ,
        0.19067776,  0.10366741, -0.19931173,  0.17700118, -0.30707324,
       -0.48089832,  0.5382253 , -0.1121833 , -1.5268826 , -0.31

In [127]:
model.get_nearest_neighbors("~", 10)

[(0.879124641418457, '~~'),
 (0.8207853436470032, '~~!'),
 (0.8176555633544922, '~~~^^'),
 (0.8164969086647034, '~~~~^^'),
 (0.8112945556640625, '~~`'),
 (0.8066441416740417, '~~~!'),
 (0.802881121635437, '^^!!'),
 (0.7944669723510742, '!!!^^'),
 (0.7887829542160034, '~!!^^'),
 (0.7870693802833557, '!~^^')]

In [88]:
model.get_nearest_neighbors(j2hcj(h2j('사랑해')), 10)

[(0.8034389615058899, 'ㅅㅏㄹㅏㅇ'),
 (0.7843306660652161, 'ㅅㅏㄹㅏㅇㅂㅣ'),
 (0.7284317016601562, 'ㅅㅏㄹㅏㅈㅕ'),
 (0.6977867484092712, 'ㅅㅏㄻㅇㅏ'),
 (0.6959367394447327, 'ㅅㅏㄹㅏ'),
 (0.684650182723999, 'ㅅㅏㄹㅏㅁㄱㅘ'),
 (0.6789019703865051, 'ㅅㅏㄹㅏㅇㄱㅘㅈㅓㄴㅈㅐㅇ'),
 (0.6772234439849854, 'ㅅㅏㄹㅏㅇㅅㅡㄹㅓㅇㅝ'),
 (0.6760623455047607, 'ㅅㅏㄹㅎㅐ'),
 (0.675919234752655, 'ㅁㅐㅎㅐ')]

# 자모단위 학습

In [53]:
import re

# 정규식 패턴: 같은 문자가 3번 이상 반복되는 경우
def reduce_repeated_chars(s):
    pattern = r'(.)\1{2,}'
    return re.sub(pattern, r'\1\1\1', s)

In [62]:
# 자모 단위 학습
from jamo import h2j, j2hcj
s = '재밌다 ㅋㅋㅋㅋㅋㅋㅋㅋ ㄱ\-ㅈ\- test..... !+ 테스트당 아'
reduced_char = reduce_repeated_chars(s)
words = okt.morphs(reduced_char, norm=True)
processed_words = [word for word in words if word not in stopwords]
jamo_list_joined = " ".join([j2hcj(h2j(word)) for word in processed_words])
jamo_list_joined

'ㅈㅐㅁㅣㅆㄷㅏ ㅋㅋㅋ ㄱ \\- ㅈ \\- test ... !+ ㅌㅔㅅㅡㅌㅡ ㄷㅏㅇ'

In [70]:
jamo_fasttext_data = './datasets/ratings_fasttext_jamo.txt'

In [73]:
with open(jamo_fasttext_data, 'a', encoding='utf-8') as f:
    for s in df['text']:
        reduced_char = reduce_repeated_chars(s)
        words = okt.morphs(reduced_char, norm=True)
        processed_words = [word for word in words if word not in stopwords]
        jamo_list_joined = " ".join([j2hcj(h2j(word)) for word in processed_words])
        f.write(jamo_list_joined + '\n')